# **Prepare Environment and Install Dependencies**

In [6]:
# Check python version
!python --version

Python 3.10.12


In [11]:
# Install dependencies
!pip install fastparquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.1 MB/s eta 0:00:00


In [8]:
!ls tlc_trip_data

yellow_tripdata_2023-01.parquet  yellow_tripdata_2023-02.parquet


# **Q1. Downloading the data**

In [40]:
# Read parquet file using pandas and process
import pandas as pd
yellow_january_data = pd.read_parquet('tlc_trip_data/yellow_tripdata_2023-01.parquet', engine='fastparquet')
yellow_february_data = pd.read_parquet('tlc_trip_data/yellow_tripdata_2023-02.parquet', engine='fastparquet')
row_jan, col_jan = yellow_january_data.shape
row_feb, col_feb = yellow_february_data.shape
print(f"Columuns length of January data: {col_jan}.")
print(f"Columuns length of February data: {col_feb}.")

Columuns length of January data: 19.
Columuns length of February data: 19.


In [41]:
yellow_january_data

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


# **Q2. Computing duration**

In [47]:
# Calculate the trip duration and its standard deviation.
# Find the date time difference between dropoff and pickup.
yellow_january_data["duration"] = pd.to_datetime(yellow_january_data["tpep_dropoff_datetime"]) - pd.to_datetime(yellow_january_data["tpep_pickup_datetime"])
# Convert it to minutes and take the ones greater than zero.
yellow_january_data["duration"] = yellow_january_data["duration"].dt.total_seconds() / 60
yellow_january_data_slice = yellow_january_data[yellow_january_data["duration"] > 0]

jan_std = yellow_january_data_slice["duration"].std()
print(f"Standard deviation of January Yellow Taxi Trip Duration is: {jan_std}")

Standard deviation of January Yellow Taxi Trip Duration is: 42.6010710644285


# **Q3. Dropping outliers**

In [62]:
print(f'Before dropping outliers. Size of df is: {yellow_january_data_slice["duration"].shape[0]}')
yellow_january_data_slice_drop = yellow_january_data[(yellow_january_data["duration"] >= 1) & (yellow_january_data["duration"] <=60)]
print(f'After dropping outliers. Size of df is: {yellow_january_data_slice_drop["duration"].shape[0]}')
# Result
fraction = yellow_january_data_slice_drop["duration"].shape[0] / yellow_january_data_slice["duration"].shape[0]
print(f'Fraction of the records left after dropped the outliers: %{fraction}')

Before dropping outliers. Size of df is: 3065645
After dropping outliers. Size of df is: 3009173
Fraction of the records left after dropped the outliers: %0.9815790804219015


# **Q4. One-hot encoding**